# EEVE Fine-Tuning



In [1]:
import huggingface_hub
huggingface_hub.login("")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Gamzadole\.cache\huggingface\token
Login successful


In [2]:
base_model = "yanolja/EEVE-Korean-10.8B-v1.0"
kr_dataset = "Bingsu/ko_alpaca_data"

new_model = "Llama3-Ko-3-8B-fine_tuning_jo"

In [2]:
import os
import torch
import transformers
from datasets import load_from_disk
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer

In [3]:
BASE_MODEL = "yanolja/EEVE-Korean-10.8B-v1.0"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, load_in_4bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
prompt = "건강을 유지하기 위한 세 가지 팁을 알려주세요."

# 텍스트 생성을 위한 파이프라인 설정
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256) # max_new_tokens: 생성할 최대 토큰 수
outputs = pipe(
    prompt,
    do_sample=True, # 샘플링 전략 사용. 확률 분포를 기반으로 다음 토큰을 선택
    temperature=0.2, # 샘플링의 다양성을 조절하는 파라미터. 값이 높을수록 랜덤성 증가
    top_k=50, # 다음 토큰을 선택할 때 상위 k개의 후보 토큰 중에서 선택. 여기에서는 상위 50개의 후보 토큰 중에서 샘플링
    top_p=0.95, # 누적 확률이 p가 될 때까지 후보 토큰을 포함
    repetition_penalty=1.2, # 반복 패널티를 적용하여 같은 단어나 구절이 반복되는 것 방지
)
print(outputs[0]["generated_text"][len(prompt):]) # 입력 프롬프트 이후에 생성된 텍스트만 출력

c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\bitsandbytes\nn\modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(



1) 매일 아침, 저녁에 반드시 양치질을 하고 치간 칫솔이나 치실을 사용하세요.
2) 정기적으로 스케일링을 받으세요.
3) 술과 담배는 피하고 음식물을 섭취한 후엔 바로 입안을 물로 충분히 적셔주세요.
Q5. 마지막으로 구강관리와 관련된 한마디 부탁드립니다.
구강질환은 예방이 가장 중요합니다. 정기적인 검진을 통해 자신의 상태를 파악하고 적절히 대처하는 것이 좋습니다.


In [4]:
import os
import torch
import transformers
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
BASE_MODEL = "yanolja/EEVE-Korean-10.8B-v1.0"
dataset_koalpaca = load_dataset(kr_dataset)
dataset_koalpaca

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 49620
    })
})

In [6]:
# 데이터프레임으로 변환
df_koalpaca = pd.DataFrame(dataset_koalpaca['train'])

# 중복 제거
df_koalpaca = df_koalpaca.drop_duplicates(keep='first', ignore_index=True)

# HuggingFace Dataset 형태로 변환
dataset_koalpaca = Dataset.from_pandas(df_koalpaca)

print(dataset_koalpaca[0])

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.', 'input': '', 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'}


In [7]:
# NF4 양자화를 위한 설정
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True, # 모델을 4비트 정밀도로 로드
    bnb_4bit_quant_type="nf4", # 4비트 NormalFloat 양자화: 양자화된 파라미터의 분포 범위를 정규분포 내로 억제하여 정밀도 저하 방지
    bnb_4bit_use_double_quant=True, # 이중 양자화: 양자화를 적용하는 정수에 대해서도 양자화 적용
    bnb_4bit_compute_dtype=torch.bfloat16 # 연산 속도를 높이기 위해 사용 (default: torch.float32)
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=nf4_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(base_model)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
prompt_input_template = """아래는 작업을 설명하는 지시사항과 추가 정보를 제공하는 입력이 짝으로 구성됩니다. 이에 대한 적절한 응답을 작성해주세요.

### 지시사항:
{instruction}

### 입력:
{input}

### 응답:"""


prompt_no_input_template = """아래는 작업을 설명하는 지시사항입니다. 이에 대한 적절한 응답을 작성해주세요.

### 지시사항:
{instruction}

### 응답:"""

In [9]:
def generate_prompt(data_point):
  instruction = data_point["instruction"]
  input = data_point["input"]
  output = data_point["output"]

  if input:
    res = prompt_input_template.format(instruction=instruction, input=input)
  else:
    res = prompt_no_input_template.format(instruction=instruction)

  if output:
    res = f"{res}{output}<|im_end|>" # eos_token을 마지막에 추가

  data_point['text'] = res

  return data_point

# 데이터셋에 프롬프트 적용
remove_column_keys = dataset_koalpaca.features.keys() # 기존 컬럼(instruction, output 등) 제거
dataset_cvted = dataset_koalpaca.select(range(1)).shuffle().map(generate_prompt, remove_columns=remove_column_keys)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [10]:
for i in range(1):
    if "건강" in dataset_cvted[i]["text"]:
        print(dataset_cvted[i]["text"])

아래는 작업을 설명하는 지시사항입니다. 이에 대한 적절한 응답을 작성해주세요.

### 지시사항:
건강을 유지하기 위한 세 가지 팁을 알려주세요.

### 응답:세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.<|im_end|>


In [11]:
def tokenize_function(examples):
  outputs = tokenizer(examples["text"], truncation=True, max_length=512)
  return outputs

remove_column_keys = dataset_cvted.features.keys()
dataset_tokenized = dataset_cvted.map(tokenize_function, batched=True, remove_columns=remove_column_keys)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [12]:
lora_config = LoraConfig(
    r=4, # LoRA 가중치 행렬의 rank. 정수형이며 값이 작을수록 trainable parameter가 적어짐
    lora_alpha=8, # LoRA 스케일링 팩터. 추론 시 PLM weight와 합칠 때 LoRA weight의 스케일을 일정하게 유지하기 위해 사용
    lora_dropout=0.05,
    bias='none', # bias 파라미터를 학습시킬지 지정. ['none', 'all', 'lora_only']
    task_type=TaskType.CAUSAL_LM
)

# 양자화된 모델을 학습하기 전, 전처리를 위해 호출
model = prepare_model_for_kbit_training(model)
# LoRA 학습을 위해서는 아래와 같이 peft를 사용하여 모델을 wrapping 해주어야 함
model = get_peft_model(model, lora_config)

# 학습 파라미터 확인
model.print_trainable_parameters()

trainable params: 2,555,904 || all params: 10,807,480,320 || trainable%: 0.0236


In [13]:
def collate_fn(examples):
    examples_batch = tokenizer.pad(examples, padding='longest', return_tensors='pt')
    examples_batch['labels'] = examples_batch['input_ids'] # 모델 학습 평가를 위한 loss 계산을 위해 입력 토큰을 레이블로 사용
    return examples_batch

train_args = transformers.TrainingArguments(
    per_device_train_batch_size=4, # 각 디바이스당 배치 사이즈. 작을수록(1~2) 좀 더 빠르게 alignment 됨
    gradient_accumulation_steps=1, 
    max_steps=-1, 
    learning_rate=1e-3, # 학습률
    bf16=True, # bf16 사용 (지원되는 하드웨어 확인 필요)
    optim="paged_adamw_8bit", # 8비트 AdamW 옵티마이저
    logging_steps=1, # 로깅 빈도
    output_dir = "./output",
    num_train_epochs=10, # epoch
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_tokenized,
    args=train_args,
    dataset_text_field="text",
    data_collator=collate_fn
)

c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\transformers\training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\trl\trainer\sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\trl\trainer\sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed w

In [14]:
model.config.use_cache = False

trainer.train()

  0%|          | 0/10 [00:00<?, ?it/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_pr

{'loss': 2.3856, 'grad_norm': 2.5717580318450928, 'learning_rate': 0.0009000000000000001, 'epoch': 1.0}
{'loss': 1.79, 'grad_norm': 3.1512644290924072, 'learning_rate': 0.0008, 'epoch': 2.0}
{'loss': 1.5235, 'grad_norm': 6.877908706665039, 'learning_rate': 0.0007, 'epoch': 3.0}
{'loss': 0.9257, 'grad_norm': 4.507090091705322, 'learning_rate': 0.0006, 'epoch': 4.0}
{'loss': 0.5123, 'grad_norm': 2.57369065284729, 'learning_rate': 0.0005, 'epoch': 5.0}
{'loss': 0.2707, 'grad_norm': 2.146669387817383, 'learning_rate': 0.0004, 'epoch': 6.0}
{'loss': 0.2125, 'grad_norm': 4.079265117645264, 'learning_rate': 0.0003, 'epoch': 7.0}
{'loss': 0.1481, 'grad_norm': 1.5603268146514893, 'learning_rate': 0.0002, 'epoch': 8.0}
{'loss': 0.1193, 'grad_norm': 0.25185197591781616, 'learning_rate': 0.0001, 'epoch': 9.0}
{'loss': 0.1159, 'grad_norm': 0.19507254660129547, 'learning_rate': 0.0, 'epoch': 10.0}
{'train_runtime': 19.483, 'train_samples_per_second': 0.513, 'train_steps_per_second': 0.513, 'train_lo

TrainOutput(global_step=10, training_loss=0.8003613464534283, metrics={'train_runtime': 19.483, 'train_samples_per_second': 0.513, 'train_steps_per_second': 0.513, 'total_flos': 45963539251200.0, 'train_loss': 0.8003613464534283, 'epoch': 10.0})

In [15]:
FINETUNED_MODEL = "eeve"
trainer.model.save_pretrained(FINETUNED_MODEL)

c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
from peft import PeftConfig

FINETUNED_MODEL = "eeve"

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True, # 모델을 4비트 정밀도로 로드
    bnb_4bit_quant_type="nf4", # 4비트 NormalFloat 양자화: 양자화된 파라미터의 분포 범위를 정규분포 내로 억제하여 정밀도 저하 방지
    bnb_4bit_use_double_quant=True, # 이중 양자화: 양자화를 적용하는 정수에 대해서도 양자화 적용
    bnb_4bit_compute_dtype=torch.bfloat16 # 연산 속도를 높이기 위해 사용 (default: torch.float32)
)

peft_config = PeftConfig.from_pretrained(FINETUNED_MODEL)
print(peft_config.base_model_name_or_path)

# 베이스 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    quantization_config=nf4_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    peft_config.base_model_name_or_path
)

# QLoRA 모델 로드
peft_model = PeftModel.from_pretrained(model, FINETUNED_MODEL, torch_dtype=torch.bfloat16)
# QLoRA 가중치를 베이스 모델에 병합
merged_model = peft_model.merge_and_unload()

yanolja/EEVE-Korean-10.8B-v1.0


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\peft\tuners\lora\bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [4]:
prompt = "건강을 유지하기 위한 세 가지 팁을 알려주세요."

# 텍스트 생성을 위한 파이프라인 설정
pipe = pipeline(task="text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=256)
outputs = pipe(
    prompt,
    do_sample=True, # 샘플링 전략 사용. 확률 분포를 기반으로 다음 토큰을 선택
    temperature=0.2, 
    top_k=50, # 다음 토큰을 선택할 때 상위 k개의 후보 토큰 중에서 선택. 여기에서는 상위 50개의 후보 토큰 중에서 샘플링
    top_p=0.95, # 누적 확률이 p가 될 때까지 후보 토큰을 포함
    repetition_penalty=1.2, # 반복 패널티를 적용하여 같은 단어나 구절이 반복되는 것 방지
)
print(outputs[0]["generated_text"])

c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


건강을 유지하기 위한 세 가지 팁을 알려주세요.
A: 첫째, 항상 좋은 자세를 유지하세요. 둘째, 운동은 필수입니다! 마지막으로, 충분한 수면을 취하는 것이 중요합니다.
